# Process BenchExec Experiments

This notebook post-processes the results obtained from [Benchexec](https://github.com/sosy-lab/benchexec) tables.

The Benchexec tables contain sets of (repeated) stat columns, one per run set. Each run set can represent a specific solver. Each row is a problem instance, called a _task_ in Benchexec.

This set-up is not very useful to do plotting or stat tables. Instead, we would like to re-shape and have a special column that records the solver, and  so one set of stat columns (with no repetition).

This network will then produce two tables:

1. A flat table of stats, that can be used for further analysis and plotting, with the solver being recorded in a new column.
2. A coverage table per domain and solver, typically reported in papers.

Sebastian Sardina 2024 - ssardina@gmail.com

In [33]:
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
import glob
import re

# sys.dont_write_bytecode = True  # prevent creation of .pyc files

CSV_FOLDER = "stats/ecai23-redo-benchexec-jul24/"
NAME_EXPERIMENT = "cfond"

## 1. Flatten Benchexec CSV tables

Collect all CSV benchexec table result files under `CSV_FOLDER` folder.

In [34]:
# files produced when tables come from single benchexec run (one .xml.bz2 file)
benchexec_csv_files = glob.glob(os.path.join(CSV_FOLDER, "**", "benchmark-*.csv"), recursive=True)

# files produced when tables come from multiple benchexec runs (many .xml.bz2 files)
benchexec_csv_files += glob.glob(
    os.path.join(CSV_FOLDER, "**", "results.*.table.csv"), recursive=True
)

print(f"CSV benchexec table files found to extract and combine stats in folder {CSV_FOLDER}:")
benchexec_csv_files

CSV benchexec table files found to extract and combine stats in folder stats/ecai23-redo-benchexec-jul24/:


['stats/ecai23-redo-benchexec-jul24/prp-19-07-24/benchmark-prp.2024-07-19_17-59-23.results.prp.FOND.csv',
 'stats/ecai23-redo-benchexec-jul24/paladinus-19-07-24/benchmark-paladinus.2024-07-19_07-50-23.results.paladinus.FOND.csv',
 'stats/ecai23-redo-benchexec-jul24/cfondasp-21-07-24/results.2024-07-29_15-40-06.table.csv',
 'stats/ecai23-redo-benchexec-jul24/fondsat-21-07-24/results.2024-07-29_15-40-20.table.csv']

Next, **load all CSV files into a single dataframe**. As explained above, each CSV file may include results over many _run sets_, with each having its own (set of) columns.

Each row is the result of a _task_ in the experiment, and every run set has its columns stats for such task. Presumably sets of columns share the same schema.

We then need reshape this structure to have just one set of columns and a new column identifying the run set. So each original row will be expanded into many rows, one per run set.

The first three lines contain header:

1. First line contains the **_tool_** used. It starts with `tool` followed by the name of the tool repeated multiple times (to match no. of columns).
2. Second line contains the **_runs_** of the experiment. It starts with `run set` and then sets of columns with the name of the runs.
3. Third line contains the **_stats_** column names repeated per run set. First column is for name of the task.

In [35]:
RENAME_COLS = {"benchmarks/benchexe/tasks/": "id", "cputime (s)": "cputime", "walltime (s)": "walltime", "memory (MB)": "memory_mb"}

def get_meta_csv(file):
    """Given a benchexec CSV file, extract the runs (e.g., prp, prp_inv) and how many columns per run"""
    with open(file, "r") as f:
        # first line contains the tool used (repeated one per column needed); e.g.,  PP-FOND
        tools_header = f.readline().split()[1:]
        # second line contains the run/solvers used in the experiment (e.g., prp, prp_inv) and starts with "run set" to be ignored - will have duplicates, one per stat col in run
        runs_header = f.readline().split()[2:]

    # get the run names (e.g., prp, prp_inv), drop duplicates but preserve order (order of columns in CSV)
    # OBS: cannot just use set() as that will change the order of columns in CSV
    runs = list(dict.fromkeys(runs_header))

    # no of stat columns per run
    no_cols = int(len(runs_header) / len(runs))

    return runs, no_cols


# iterate through each CSV file
dfs = []
for f in benchexec_csv_files:
    runs, no_cols = get_meta_csv(f)
    print(f"Runs in file {f}: {runs} with {no_cols} stat columns")

    # go over each set of run columns (a CSV table file may contain several runs, each with the same columns)
    for k, r in enumerate(list(runs)):
        col_idx = [0] + list(range(k*no_cols + 1, k*no_cols + no_cols + 1))
        print(f"\t Extracting run '{r}' in columns: {col_idx}")

        # read the CSV file from line 3+ (line 3 is header)
        df = pd.read_csv(f, delimiter="\t", skiprows=2, usecols=col_idx)
        df.rename(columns=lambda x: x.split('.')[0], inplace=True)

        df.columns.values[0] = "task"
        # df.rename(columns={df.columns[1]: "task"})

        # populate column run with name of run-solver r
        df.insert(1, 'run', r)
        dfs.append(df)

df_csv = pd.concat(dfs).reset_index(drop=True)

df_csv.rename(columns=RENAME_COLS, inplace=True)

print("Runs found:", df_csv["run"].unique())
# df.set_index("task", inplace=True)


# df_csv.query("task == 'acrobatics_01.yml' and run == 'cfondasp1-reg.FOND'")
df_csv

Runs in file stats/ecai23-redo-benchexec-jul24/prp-19-07-24/benchmark-prp.2024-07-19_17-59-23.results.prp.FOND.csv: ['prp.FOND'] with 6 stat columns
	 Extracting run 'prp.FOND' in columns: [0, 1, 2, 3, 4, 5, 6]
Runs in file stats/ecai23-redo-benchexec-jul24/paladinus-19-07-24/benchmark-paladinus.2024-07-19_07-50-23.results.paladinus.FOND.csv: ['paladinus.FOND'] with 6 stat columns
	 Extracting run 'paladinus.FOND' in columns: [0, 1, 2, 3, 4, 5, 6]
Runs in file stats/ecai23-redo-benchexec-jul24/cfondasp-21-07-24/results.2024-07-29_15-40-06.table.csv: ['cfondasp1-fsat.FOND', 'cfondasp1-reg.FOND', 'cfondasp2-fsat.FOND', 'cfondasp2-reg.FOND'] with 6 stat columns
	 Extracting run 'cfondasp1-fsat.FOND' in columns: [0, 1, 2, 3, 4, 5, 6]
	 Extracting run 'cfondasp1-reg.FOND' in columns: [0, 7, 8, 9, 10, 11, 12]
	 Extracting run 'cfondasp2-fsat.FOND' in columns: [0, 13, 14, 15, 16, 17, 18]
	 Extracting run 'cfondasp2-reg.FOND' in columns: [0, 19, 20, 21, 22, 23, 24]
Runs in file stats/ecai23-re

,task,run,status,cputime,walltime,memory_mb,planner_time,policy_size
0,acrobatics_01.yml,prp.FOND,true,0.233342,0.235622,10.080256,0.01,4
1,acrobatics_02.yml,prp.FOND,true,10.369209,10.374620,10.199040,10.14,8
2,acrobatics_03.yml,prp.FOND,true,11.289586,11.302472,10.235904,11.05,16
3,acrobatics_04.yml,prp.FOND,true,12.004272,12.007289,10.444800,11.76,32
4,acrobatics_05.yml,prp.FOND,true,17.225556,17.242667,11.210752,16.95,64
...,...,...,...,...,...,...,...,...
4715,zenotravel_11.yml,fondsat-glucose.FOND,TIMEOUT (false),14400.263172,14401.003335,2337.935360,-1.00,-1
4716,zenotravel_12.yml,fondsat-glucose.FOND,TIMEOUT (false),14401.220999,14402.025636,2084.392960,-1.00,-1
4717,zenotravel_13.yml,fondsat-glucose.FOND,TIMEOUT (false),14400.344840,14401.002512,1823.010816,-1.00,-1
4718,zenotravel_14.yml,fondsat-glucose.FOND,TIMEOUT (false),14401.257835,14403.561401,2396.045312,-1.00,-1


We next **enrich** the dataframe with the following derived columns:

* domain
* instance
* solver
* solved (boolean)

In [36]:
def get_benchmark_labels(task_name):
    """From the task description name (e.g., acrobatics_01.yml), extract the benchmark labels, like domain, instance"""
    regex = r"(.+)_([0-9]+)\.yml"

    match = re.match(regex, task_name)
    if match:
        # print(match.groups())
        domain = match.group(1)
        instance = match.group(2)
    else:
        print("Problem extracting labels from task name", task_name)
    return domain, instance

df = df_csv.copy()

# 1 - split task name into domain and instances
df["benchmark"] = df.reset_index()["task"].map(get_benchmark_labels).values
df["domain"] = df["benchmark"].str.get(0)
df["instance"] = df["benchmark"].str.get(1)
df.drop(columns=["benchmark"], inplace=True)

# 2 - map status from benchexec to integers status
map_status = {
    "true": 1,
    "false": 0,
    "True": 1,
    "False": 0,
    False: 0,
    True: 1,
    "OUT OF MEMORY (false)": -2,
    "TIMEOUT (false)": -1,
    "TIMEOUT (true)": 1,
}
df["status2"] = df["status"].map(map_status)

missing_mapping = df[df["status2"].isnull()].shape[0]
if missing_mapping > 0:
    missing_status = [x for x in df["status"].unique() if x not in map_status.keys()]
    print(f"WARNING: {missing_mapping} status values not mapped:", missing_status)
    print(df[df["status2"].isnull()])

df["status"] = df["status2"]
df.drop(columns=["status2"], inplace=True)

# 3 - define Boolean column solved to flag if solved or not based on status
df.insert(3, "solved", df["status"].apply(lambda x: True if x == 1 else False))


# 4 - extract solver from run name
map_solver = {
    "prp.FOND": "prp",
    "paladinus.FOND": "paladinus",
    "fondsat-glucose.FOND": "fondsat-glucose",
    "fondsat-minisat.FOND": "fondsat-minisat",
    "cfondasp1-reg.FOND" : "cfondasp1-reg",
    "cfondasp2-reg.FOND" : "cfondasp2-reg",
    "cfondasp1-fsat" : "asp1-fsat",
    "cfondasp2-fsat" : "asp2-fsat",
}
df["solver"] = df["run"].map(map_solver)

# sanity check status
# df.query("status not in [-1,0,-2,1]")
# df.status = df.status.astype(int) # convert to int
# df.loc[df.status == "OUT OF MEMORY (false)"]
# df.loc[df.status == -1]

# df.dtypes

# note that status should be integer; if float it is bc there must be NaN value!
df

,task,run,status,solved,cputime,walltime,memory_mb,planner_time,policy_size,domain,instance,solver
0,acrobatics_01.yml,prp.FOND,1,True,0.233342,0.235622,10.080256,0.01,4,acrobatics,01,prp
1,acrobatics_02.yml,prp.FOND,1,True,10.369209,10.374620,10.199040,10.14,8,acrobatics,02,prp
2,acrobatics_03.yml,prp.FOND,1,True,11.289586,11.302472,10.235904,11.05,16,acrobatics,03,prp
3,acrobatics_04.yml,prp.FOND,1,True,12.004272,12.007289,10.444800,11.76,32,acrobatics,04,prp
4,acrobatics_05.yml,prp.FOND,1,True,17.225556,17.242667,11.210752,16.95,64,acrobatics,05,prp
...,...,...,...,...,...,...,...,...,...,...,...,...
4715,zenotravel_11.yml,fondsat-glucose.FOND,-1,False,14400.263172,14401.003335,2337.935360,-1.00,-1,zenotravel,11,fondsat-glucose
4716,zenotravel_12.yml,fondsat-glucose.FOND,-1,False,14401.220999,14402.025636,2084.392960,-1.00,-1,zenotravel,12,fondsat-glucose
4717,zenotravel_13.yml,fondsat-glucose.FOND,-1,False,14400.344840,14401.002512,1823.010816,-1.00,-1,zenotravel,13,fondsat-glucose
4718,zenotravel_14.yml,fondsat-glucose.FOND,-1,False,14401.257835,14403.561401,2396.045312,-1.00,-1,zenotravel,14,fondsat-glucose


Finally, save all results into a complete CSV file. This file can be later used to plot time-coverage graphs as in Nitin's R script.

These tables are not flatten, runs are not across column sets but there is a designated column `solver` that specifies the run of the row.

In [37]:
df.to_csv(os.path.join(CSV_FOLDER, f"{NAME_EXPERIMENT}_benchexec_stats.csv"), index=False)

## 2. Coverage Analysis and Table

We now generate **coverage** tables, as they often appear in papers. Basically, we compute the following per domain and solver-run:

- **Coverage:** % of solved instances solved by the solver-run; and
- **Stat metrics:** mean on time, memory usage, and policy size.

In [38]:
print(df.shape)
df.head()

(4720, 12)


,task,run,status,solved,cputime,walltime,memory_mb,planner_time,policy_size,domain,instance,solver
0,acrobatics_01.yml,prp.FOND,1,True,0.233342,0.235622,10.080256,0.01,4,acrobatics,01,prp
1,acrobatics_02.yml,prp.FOND,1,True,10.369209,10.374620,10.199040,10.14,8,acrobatics,02,prp
2,acrobatics_03.yml,prp.FOND,1,True,11.289586,11.302472,10.235904,11.05,16,acrobatics,03,prp
3,acrobatics_04.yml,prp.FOND,1,True,12.004272,12.007289,10.444800,11.76,32,acrobatics,04,prp
4,acrobatics_05.yml,prp.FOND,1,True,17.225556,17.242667,11.210752,16.95,64,acrobatics,05,prp


Calculate % ratio per set/domain/sub_domain/run-solver.

In [39]:
df_grouped = df.groupby(["domain", "solver"])

#   df_grouped.sum()[["solved"]] = sum all the True instances (sum over bool = number of True)
#   df_grouped.count()[["solved"]] = number of rows in solved column (includes True and False values)
df_coverage = (
    df_grouped.sum(numeric_only=True)[["solved"]] / df_grouped.count()[["solved"]]
)
df_coverage

solved
domain     solver                   
acrobatics cfondasp1-reg    0.500000
           cfondasp2-reg    0.000000
           fondsat-glucose  0.500000
           fondsat-minisat  0.375000
           paladinus        1.000000
...                              ...
zenotravel cfondasp2-reg    0.000000
           fondsat-glucose  0.333333
           fondsat-minisat  0.333333
           paladinus        0.333333
           prp              1.000000

[102 rows x 1 columns]

Calculate mean metric (for CPU time, memory, and policy size) across the solved instances.

In [40]:
columns = ["domain", "solver", "cputime", "memory_mb", "policy_size"]
df_solved = df.query("solved == True")[columns]

df_solved_grouped = df_solved.groupby(["domain", "solver"])
df_metrics = df_solved_grouped.mean()
df_metrics

cputime   memory_mb  policy_size
domain     solver                                              
acrobatics cfondasp1-reg    688.996671   80.991232    15.000000
           fondsat-glucose  178.673675   82.584576    15.000000
           fondsat-minisat    7.889886   23.134208     9.333333
           paladinus         11.176993   55.968768   126.500000
           prp               13.011640   12.451840   127.500000
...                                ...         ...          ...
zenotravel cfondasp1-reg    940.044956  179.512320    15.500000
           fondsat-glucose  170.958383  466.844058    13.000000
           fondsat-minisat  406.648567  397.922304    13.000000
           paladinus         16.789707  368.121446    15.200000
           prp                3.753867   66.439305    54.600000

[85 rows x 3 columns]

Put together **Coverage** and **Metrics** tables.

In [41]:
column_names = {
    "solved": "cov",
    "cputime": "time",
    "memory_mb": "mem",
    "policy_size": "size",
}

df_stats = df_coverage.join(df_metrics, how="inner")
df_stats.rename(columns=column_names, inplace=True)

df_stats = df_stats.reset_index()

df_stats

,domain,solver,cov,time,mem,size
0,acrobatics,cfondasp1-reg,0.500000,688.996671,80.991232,15.000000
1,acrobatics,fondsat-glucose,0.500000,178.673675,82.584576,15.000000
2,acrobatics,fondsat-minisat,0.375000,7.889886,23.134208,9.333333
3,acrobatics,paladinus,1.000000,11.176993,55.968768,126.500000
4,acrobatics,prp,1.000000,13.011640,12.451840,127.500000
...,...,...,...,...,...,...
80,zenotravel,cfondasp1-reg,0.266667,940.044956,179.512320,15.500000
81,zenotravel,fondsat-glucose,0.333333,170.958383,466.844058,13.000000
82,zenotravel,fondsat-minisat,0.333333,406.648567,397.922304,13.000000
83,zenotravel,paladinus,0.333333,16.789707,368.121446,15.200000


Finally, pivot the column `solver` into (set of) columns, one per solver.

In [42]:
df_stats_pivot = df_stats.pivot(
    index=["domain"],
    values=["cov", "time", "mem", "size"],
    columns="solver",
)
df_stats_pivot.reset_index(
    inplace=True
)  # unfold multi-index into columns (create integer index)
df_stats_pivot.columns = [
    "_".join(tup).rstrip("_") for tup in df_stats_pivot.columns.values
]

# flat index, but multi-column: 1. coverage / time / policy size and 2. each solver/run
df_stats_pivot = df_stats_pivot.round(2)

df_stats_pivot

,domain,cov_cfondasp1-reg,cov_fondsat-glucose,cov_fondsat-minisat,cov_paladinus,cov_prp,time_cfondasp1-reg,time_fondsat-glucose,time_fondsat-minisat,time_paladinus,...,mem_cfondasp1-reg,mem_fondsat-glucose,mem_fondsat-minisat,mem_paladinus,mem_prp,size_cfondasp1-reg,size_fondsat-glucose,size_fondsat-minisat,size_paladinus,size_prp
0,acrobatics,0.50,0.50,0.38,1.00,1.00,689.00,178.67,7.89,11.18,...,80.99,82.58,23.13,55.97,12.45,15.00,15.00,9.33,126.50,127.50
1,beam-walk,0.27,0.27,0.27,0.82,1.00,47.08,160.18,1336.87,1.39,...,56.27,79.61,241.17,56.60,18.99,18.67,18.67,18.67,453.22,1488.73
2,blocksworld-ipc08,0.33,0.40,0.37,0.33,1.00,9.46,343.36,1031.14,2.26,...,69.54,369.89,285.50,88.17,27.00,11.20,12.17,11.64,12.00,24.40
3,blocksworld-new,0.20,0.20,0.20,0.15,1.00,1885.13,152.74,594.82,5.11,...,105.27,209.71,273.94,154.84,264.74,10.25,10.12,10.12,8.00,54.22
4,chain-of-rooms,0.10,0.10,0.10,0.20,1.00,182.30,219.46,2283.79,60.14,...,93.29,147.87,568.15,76.75,11.42,28.00,28.00,28.00,42.00,163.00
5,doors,1.00,1.00,0.87,1.00,0.80,1668.95,293.27,1666.56,1304.16,...,107.86,138.88,358.77,359.96,39.05,19.00,19.00,17.00,17473.73,22.00
6,earth_observation,0.20,0.25,0.15,0.25,1.00,2862.18,1178.06,571.36,361.16,...,70.66,144.49,49.72,66.14,10.47,16.38,19.20,11.50,30.60,148.45
7,elevators,0.47,0.53,0.47,0.47,1.00,68.53,1614.44,107.16,5.48,...,61.59,85.65,51.66,69.30,10.27,15.86,18.38,15.86,19.71,46.53
8,faults-ipc08,0.69,0.82,0.51,0.85,1.00,860.84,2147.57,334.28,10.67,...,65.81,101.73,79.09,564.63,10.44,13.92,13.87,11.25,180.51,15.18
9,first-responders-ipc08,0.44,0.52,0.46,0.30,0.75,872.98,477.87,606.29,53.99,...,70.61,102.16,145.88,510.72,12.33,9.98,11.31,10.24,9.03,18.08


Save coverage table to a CSV file, this can be used in papers.

In [43]:
df_stats_pivot.to_csv(os.path.join(CSV_FOLDER, f"{NAME_EXPERIMENT}_coverage_table.csv"), index=False)